In [2]:
pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re


excel_file = "10NOV2025_5 aM 73 min.xlsx"
n_high, n_low = 5, 5

def parse_sheet_name(name):
    """
    Parses sheet name like '5 aM_0.1' or '5 aM_15.3'.
    If no replicate (dot) found, assumes replicate = 1.
    """
    parts = name.split('_', 1)
    conc = parts[0].strip()
    if len(parts) < 2:
        return conc, 0.0, 1
    t_rep = parts[1]
    if '.' in t_rep:
        time, rep = t_rep.split('.')
        time = float(time)
        rep = int(rep)
    else:
        # if only time is present (no replicate)
        time = float(re.sub(r'[^0-9.]', '', t_rep))  # strip text like 'min'
        rep = 1
    return conc, time, rep


def estimate_rct(df, n_high=5, n_low=5):
    """Estimate Rs and Rct from Nyquist data."""
    # make sure we work with stringified column names
    cols = list(df.columns)

    freq_candidates = [c for c in cols if "freq" in str(c).lower()]
    if not freq_candidates:
        raise ValueError(f"No frequency column found. Columns: {cols}")
    freq_col = freq_candidates[0]

    zreal_candidates = [
        c for c in cols
        if ("z'" in str(c).lower()
            or "zre" in str(c).lower()
            or ("real" in str(c).lower() and "imag" not in str(c).lower()))
    ]
    if not zreal_candidates:
        raise ValueError(f"No Z' (real) column found. Columns: {cols}")
    zreal_col = zreal_candidates[0]

    df2 = df[[freq_col, zreal_col]].dropna()
    df2 = df2.sort_values(by=freq_col, ascending=False)

    n_high = min(n_high, len(df2))
    n_low = min(n_low, len(df2))

    Rs = df2[zreal_col].head(n_high).mean()
    Rt = df2[zreal_col].tail(n_low).mean()
    Rct = Rt - Rs
    return float(Rs), float(Rct)


# --- load and extract Rct per sheet ---
xls = pd.ExcelFile(excel_file)
records = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    conc, time_min, rep = parse_sheet_name(sheet)
    Rs, Rct = estimate_rct(df, n_high, n_low)
    records.append({"Time_min": time_min, "Replicate": rep, "Rct_ohm": Rct})

data = pd.DataFrame(records)

# --- average three reads per timepoint ---
mean_data = data.groupby("Time_min", as_index=False).agg(
    Rct_mean=("Rct_ohm", "mean"),
    Rct_std=("Rct_ohm", "std")
)

# --- use 0 min mean as baseline ---
Rct0 = mean_data.loc[mean_data["Time_min"] == 0, "Rct_mean"].values[0]
mean_data["Delta_Rct"] = mean_data["Rct_mean"] - Rct0
mean_data["Delta_Rct_norm_%"] = mean_data["Delta_Rct"] / Rct0 * 100

# --- save and plot ---
out_csv = excel_file.replace(".xlsx", "_timecourse.csv")
mean_data.to_csv(out_csv, index=False)
print(f"Saved summary to {out_csv}\n", mean_data)

plt.errorbar(mean_data["Time_min"], mean_data["Rct_mean"],
             yerr=mean_data["Rct_std"], fmt="o-", capsize=4)
plt.xlabel("Time (min)")
plt.ylabel("Rct (Ω)")
plt.title("5 aM – Mean Rct vs Time")
plt.tight_layout()
plt.savefig(excel_file.replace(".xlsx", "_Rct_vs_time.png"), dpi=300)
plt.show()


ValueError: No Z' (real) column found. Columns: ['5 aM_Baseline 1', '34 freqs', datetime.datetime(2025, 10, 11, 14, 1, 36), 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']